# Machine Learning Foundation

## Section 1, Part b: Reading Data Lab

In [1]:
# Imports
import sqlite3 as sq3
import pandas.io.sql as pds
import pandas as pd
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

## Lab Exercise: Reading in database files

 - Create a variable, `path`, containing the path to the `baseball.db` contained in `resources/`
 - Create a connection, `con`, that is connected to database at `path`
 - Create a variable, `query`, containing a SQL query which reads in all data from the `allstarfull` table
 - Create a variable, `observations`, by using pandas' [read_sql](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html)

### Optional
 - Create a variable, `tables`, which reads in all data from the table `sqlite_master`
 - Pretend that you were interesting in creating a new baseball hall of fame. Join and analyze the tables to evaluate the top 3 all time best baseball players.

Will be using WS Studio Notebook pull for the baseball.db file, use Credentials when loading it in

For extra help, see: https://medium.com/ibm-data-science-experience/working-with-ibm-cloud-object-storage-in-python-fe0ba8667d5f

In [2]:
# The code was removed by Watson Studio for sharing.

In [20]:
# make sure the credentials below in the def match the credentials you loaded above
def download_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('The file %s has been downloaded into this notebook' % key)

In [21]:
# For local file, input the name of what you want the file to be, for the key put name of the file on Cloud Object Storage
# it would be best if they match for ease
download_file_cos(credentials,'baseball.db','baseball.db')

The file baseball.db has been downloaded into this notebook


In [22]:
### BEGIN SOLUTION
# Create a variable, `path`, containing the path to the `baseball.db` contained in `resources/`
path = 'baseball.db'

# Create a connection, `con`, that is connected to database at `path`
con = sq3.Connection(path)

In [23]:
def sql_fetch(con):

    cursorObj = con.cursor()

    cursorObj.execute('SELECT name from sqlite_master where type= "table"')

    print(cursorObj.fetchall())

sql_fetch(con)

[('allstarfull',), ('schools',), ('batting',)]


In [24]:
# Create a variable, `query`, containing a SQL query which reads in all data from the `` table

query = """
SELECT *
FROM allstarfull;
"""

allstar_observations = pd.read_sql(query, con)
allstar_observations.head()

,index,playerID,yearID,gameNum,gameID,teamID,lgID,GP,startingPos
0,0,gomezle01,1933,0,ALS193307060,NYA,AL,1.0,1.0
1,1,ferreri01,1933,0,ALS193307060,BOS,AL,1.0,2.0
2,2,gehrilo01,1933,0,ALS193307060,NYA,AL,1.0,3.0
3,3,gehrich01,1933,0,ALS193307060,DET,AL,1.0,4.0
4,4,dykesji01,1933,0,ALS193307060,CHA,AL,1.0,5.0


In [25]:
# print(allstar_observations)

# Create a variable, tables, which reads in all data from the table sqlite_master
all_tables = pd.read_sql('SELECT * FROM sqlite_master', con)
print(all_tables)

    type                  name     tbl_name  rootpage  \
0  table           allstarfull  allstarfull         2   
1  index  ix_allstarfull_index  allstarfull         3   
2  table               schools      schools        26   
3  index      ix_schools_index      schools        31   
4  table               batting      batting        99   
5  index      ix_batting_index      batting       100   

                                                 sql  
0  CREATE TABLE "allstarfull" (\n"index" INTEGER,...  
1  CREATE INDEX "ix_allstarfull_index"ON "allstar...  
2  CREATE TABLE "schools" (\n"index" INTEGER,\n  ...  
3  CREATE INDEX "ix_schools_index"ON "schools" ("...  
4  CREATE TABLE "batting" (\n"index" INTEGER,\n  ...  
5  CREATE INDEX "ix_batting_index"ON "batting" ("...  


In [26]:
# Pretend that you were interesting in creating a new baseball hall of fame. Join and analyze the tables to evaluate the top 3 all time best baseball players
best_query = """
SELECT playerID, sum(GP) AS num_games_played, AVG(startingPos) AS avg_starting_position
    FROM allstarfull
    GROUP BY playerID
    ORDER BY num_games_played DESC, avg_starting_position ASC
    LIMIT 3
"""
best = pd.read_sql(best_query, con)
print(best.head())
### END SOLUTION

    playerID  num_games_played  avg_starting_position
0  musiast01              24.0               6.357143
1   mayswi01              24.0               8.000000
2  aaronha01              24.0               8.470588


---
### Machine Learning Foundation (C) 2020 IBM Corporation